# First Part

In [3]:
import pandas as pd
CollegeTrain = pd.read_csv("CollegeTrain.csv", index_col=0)
CollegeTest = pd.read_csv("CollegeTest.csv", index_col=0)

In [4]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

In [5]:
# Adjusting dataset and the samples
SampleTrain = CollegeTrain.sample(frac = 0.05, random_state=0)

SampleYtrain = SampleTrain['Outcome']
SampleXtrain = SampleTrain.drop(['Outcome'], axis = 1)

SampleTest = CollegeTest.sample(n=100, random_state=0)

SampleYtest = SampleTest['Outcome']
SampleXtest = SampleTest.drop(['Outcome'], axis = 1)

In [6]:
# Training the classifier
Classifier = DecisionTreeClassifier(random_state=0)
FittedClassifier = Classifier.fit(SampleXtrain, SampleYtrain)

In [7]:
Ytrain_pred = FittedClassifier.predict(SampleXtest)

In [8]:
accuracy_score(SampleYtest, Ytrain_pred)

0.64

# Second Part

In [9]:
# Sample Test
import statistics
TestXsamples =[]
TestYsamples =[]

for i in range(100):
    SampleTest = CollegeTest.sample(n=100, random_state=i)
    SampleYtest = SampleTest['Outcome']
    SampleXtest = SampleTest.drop(['Outcome'], axis = 1)
    TestXsamples.append(SampleXtest)
    TestYsamples.append(SampleYtest)
    
test_accs = []

for j in range(len(TestXsamples)):
    Ytrain_pred = FittedClassifier.predict(TestXsamples[j])
    test_accs.append(accuracy_score(TestYsamples[j], Ytrain_pred))

mean_test_acc = statistics.mean(test_accs)

In [83]:
len(test_accs)

100

In [77]:
import numpy as np
np.percentile(test_accs, 50)

0.56

# Third part

In [82]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
import statistics
import numpy as np


# Initializing arrays
indiv_test_acc =[]
CI_lower_bound = []
CI_upper_bound = []
mean_test_acc = []
observed_lower_bound = []
observed_upper_bound = []


for i in range(20):
    # Adjusting dataset and the samples
    SampleTrain = CollegeTrain.sample(frac = 0.05, random_state=i)

    SampleYtrain = SampleTrain['Outcome']
    SampleXtrain = SampleTrain.drop(['Outcome'], axis = 1)

    SampleTest = CollegeTest.sample(n=100, random_state=i)

    SampleYtest = SampleTest['Outcome']
    SampleXtest = SampleTest.drop(['Outcome'], axis = 1)

    # Training the classifier
    Classifier = DecisionTreeClassifier(random_state=0)
    FittedClassifier = Classifier.fit(SampleXtrain, SampleYtrain)

    # Test Accuracy
    Ytrain_pred = FittedClassifier.predict(SampleXtest)

    test_acc = accuracy_score(SampleYtest, Ytrain_pred)
    indiv_test_acc.append(test_acc)
    
    # Compute Bound
    S2 = ((((test_acc*100)*((test_acc-1)**2))+(((1-test_acc)*100)*(test_acc**2)))/100)
    S = S2**(1/2)
    SB = 1.96*S/10
    
    # CI_lower_bound and upper
    CI_lower_bound.append(test_acc-SB)
    CI_upper_bound.append(test_acc+SB)
    
    # 100 repetition
    TestXsamples1 =[]
    TestYsamples1 =[]
    
    for j in range(100):
        SampleTest1 = CollegeTest.sample(n=100, random_state=((i+1)*j))
        SampleYtest1 = SampleTest1['Outcome']
        SampleXtest1 = SampleTest1.drop(['Outcome'], axis = 1)
        TestXsamples1.append(SampleXtest1)
        TestYsamples1.append(SampleYtest1)
    
    test_accs1 = []
    
    for j in range(len(TestXsamples1)):
        Ytrain_pred1 = FittedClassifier.predict(TestXsamples1[j])
        test_accs1.append(accuracy_score(TestYsamples1[j], Ytrain_pred1))
    º
    mean_test_acc_i = statistics.mean(test_accs1)
    mean_test_acc.append(mean_test_acc_i)
    observed_lower_bound.append(np.percentile(test_accs1, 2.5))
    observed_upper_bound.append(np.percentile(test_accs1, 97.5))
    
frame = pd.DataFrame(columns = ["indiv_test_acc", "CI_lower_bound", "CI_upper_bound", "mean_test_acc", "observed_lower_bound", "observed_upper_bound"])
frame["indiv_test_acc"] = indiv_test_acc
frame["CI_lower_bound"] = CI_lower_bound
frame["CI_upper_bound"] = CI_upper_bound
frame["mean_test_acc"] = mean_test_acc
frame["observed_lower_bound"] = observed_lower_bound
frame["observed_upper_bound"] = observed_upper_bound

In [83]:
frame

,indiv_test_acc,CI_lower_bound,CI_upper_bound,mean_test_acc,observed_lower_bound,observed_upper_bound
0,0.64,0.545920,0.734080,0.6674,0.61000,0.73525
1,0.59,0.493601,0.686399,0.6191,0.55000,0.68000
2,0.62,0.524864,0.715136,0.6825,0.61475,0.74525
3,0.61,0.514401,0.705599,0.6718,0.59950,0.72525
4,0.63,0.535370,0.724630,0.6720,0.61000,0.73000
5,0.71,0.621063,0.798937,0.6109,0.55000,0.67000
6,0.61,0.514401,0.705599,0.5820,0.52000,0.65000
7,0.71,0.621063,0.798937,0.7273,0.67000,0.78000
8,0.67,0.577838,0.762162,0.7455,0.69475,0.80000
9,0.71,0.621063,0.798937,0.6854,0.63000,0.75525
